# RUN vert: Non-Mammalian Vertebrates

:cluck: :cluck:

```
Lead     : ababaian
Issue    : 
Version  : 0.3.1
reference: cov3m
start    : 2020 05 25
complete : 2020 05 --
files    : ~/serratus/notebook/200525_ab/
s3_files : s3://serratus-public/notebook/200525_ab/
output   : s3://serratus-public/out/200525_vert/
```

### Objectives
- Download `sraRunInfo` for all *vertebrate* RNA-seq, meta-transcriptomics and meta-genomics
- Run `Serratus` to discover new coronaviruses in these sequencing libraries


In [1]:
date

Tue May 26 15:31:18 PDT 2020


## Download sraRunInfo

SRA Accessed: 2020/05/25
Search Term: 
```
("Vertebrata"[Organism] NOT "Mammalia"[Organism] NOT "Homo sapiens"[Organism] NOT "Mus musculus"[orgn]) AND ("type_rnaseq"[Filter] OR "metagenomic"[Filter] OR "metatranscriptomic"[Filter]) AND cluster_public[prop] AND "platform illumina"[Properties]
```

Results: `86934` Accessions


## Initialize local workspace

In [2]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS

## Serratus was updated, genome remains the same
git rev-parse HEAD # commit version

# Create local run directory
WORK="$SERRATUS/notebook/200525_ab"
mkdir -p $WORK; cd $WORK

# SRA RunInfo Table base for run
RUNINFO="$WORK/vert_sraRunInfo.csv"

1450e5f18a2605ff36e98c954874169ca130ad11


In [3]:
# Randomize input list (avoid large SRA co-occuring)
cd $WORK

head -n1 $RUNINFO > vert.header
tail -n +2 $RUNINFO | shuf - > vert.rand

split -l 20000 vert.rand 

cat vert.header xaa > vert_1.csv
cat vert.header xab > vert_2.csv
cat vert.header xac > vert_3.csv
cat vert.header xad > vert_4.csv
cat vert.header xae > vert_5.csv

rm xa*
rm vert.header vert.rand

wc -l *
md5sum vert*

# Why are there 94908 entries in the download for ~86K accessions
# visible on the web browser?

   20001 vert_1.csv
   20001 vert_2.csv
   20001 vert_3.csv
   20001 vert_4.csv
   14909 vert_5.csv
   94909 vert_sraRunInfo.csv
  189822 total
e49c0208b7b07074e1fd01f42cdba81b  vert_1.csv
43044a323448a47e6edab675a1c44f8f  vert_2.csv
78b20a25210e896924474e8433565766  vert_3.csv
627d7616fd2a224d65775cf6b8fb6d53  vert_4.csv
c06a444644fd26173dd86be2b2e88981  vert_5.csv
e39b50b78465f7e12676ef18d179de5f  vert_sraRunInfo.csv


In [4]:
md5sum $RUNINFO > $RUNINFO.md5sum
###
aws s3 cp $RUNINFO s3://serratus-public/out/200525_vert/
aws s3 cp $RUNINFO.md5sum s3://serratus-public/out/200525_vert/

upload: ./vert_sraRunInfo.csv to s3://serratus-public/out/200525_vert/vert_sraRunInfo.csv
upload: ./vert_sraRunInfo.csv.md5sum to s3://serratus-public/out/200525_vert/vert_sraRunInfo.csv.md5sum


### Clean-up Run - SRA errors be here


In [65]:
# Create a list of all completed runs to date
cd $WORK
RUNINFO=$WORK/vert_sraRunInfo.csv
BATCH='batchB'
S3_PATH="s3://serratus-public/out/200525_vert/summary/"


aws s3 ls $S3_PATH > $BATCH.complete
cat $BATCH.complete | sed 's/^...............................//g' - | cut -f1 -d'.' - > $BATCH.sra.complete

cd $WORK

wc -l $RUNINFO
wc -l $BATCH.sra.complete

#sed -i 's/$/,/g' $BATCH.sra.complete

grep -vif $BATCH.sra.complete $RUNINFO > "$BATCH"_sraRunInfo.csv

wc -l  "$BATCH"_sraRunInfo.csv

94909 /home/artem/serratus/notebook/200525_ab/vert_sraRunInfo.csv
67475 batchB.sra.complete
27434 batchB_sraRunInfo.csv


In [74]:
# Create a list of all completed runs to date
cd $WORK
RUNINFO=$WORK/vert_sraRunInfo.csv
BATCH='batchC'
S3_PATH="s3://serratus-public/out/200525_vert/summary/"


aws s3 ls $S3_PATH > $BATCH.complete
cat $BATCH.complete | sed 's/^...............................//g' - | cut -f1 -d'.' - > $BATCH.sra.complete

cd $WORK

wc -l $RUNINFO
wc -l $BATCH.sra.complete

#sed -i 's/$/,/g' $BATCH.sra.complete

grep -vif $BATCH.sra.complete $RUNINFO > "$BATCH"_sraRunInfo.csv

wc -l  "$BATCH"_sraRunInfo.csv

CURRENT_BATCH=$WORK/"$BATCH"_sraRunInfo.csv




94909 /home/artem/serratus/notebook/200525_ab/vert_sraRunInfo.csv
82469 batchC.sra.complete
12431 batchC_sraRunInfo.csv


# Vertebrate Run -- 1


In [5]:
cd $WORK
CURRENT_BATCH=$WORK/vert_1.csv

echo $CURRENT_BATCH

/home/artem/serratus/notebook/200525_ab/vert_1.csv


## Running Serratus 
Upload the run data, scale-out the cluster, monitor performance.

### Terraform Initialization



In [62]:
# Terraform customization
# For a larger run beef up everything!!
# For 200527 updated align node to c5.xlarge (4 vCPU)
git diff $SERRATUS/terraform/main/main.tf

diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index 32f14e7..5fca110 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -101,7 +101,7 @@ module "monitoring" {
   security_group_ids = [aws_security_group.internal.id]
   key_name           = var.key_name
   scheduler_ip       = module.scheduler.private_ip
-  instance_type      = "r5.large"
+  instance_type      = "r5.xlarge"
 }
 
 // Serratus-dl
@@ -109,13 +109,13 @@ module "download" {
   source             = "../worker"
 
   desired_size       = 0
-  max_size           = 250
+  max_size           = 300
 
   dev_cidrs          = var.dev_cidrs
   security_group_ids = [aws_security_group.internal.id]
 
   instance_type      = "r5.large" // Mitigate the memory leak in fastq-dump
-  volume_size        = 50 // Mitigate the storage leak in fastq-dump
+  volume_size        = 200 // Mitigate the storage leak in fastq-dump
   spot_price         = 0.05
 
   s3_bucket          = module.work_bucket.name
@@ -134,7 +13

In [78]:
# Initialize terraform
TF=$SERRATUS/terraform/main
cd $TF
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

Initializing modules...

Initializing the backend...

Initializing provider plugins...

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.
module.download.data.aws_ami.amazon_linux_2: Refreshing state...
module.merge.data.aws_region.current: Refreshing state...
module.scheduler.data.aws_region.current: Refreshing state...
module.align.data.aws_region.current: Refreshing state...
module.align.data.aws_availability_zones.all: Refreshing state...
module.download.data.aws_availability_zones.all: Refreshing state...
module.download.data.aws_region.current: Refreshing state...
module.scheduler.data.aws_ami.amazon_

### Run Monitors & Upload table
Open SSH tunnels to monitor node then open monitors in browser


In [79]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>
#

Tunnels created:

    localhost:3000 -- grafana

    localhost:9090 -- prometheus

    localhost:8000 -- scheduler


### vert with cov3m

In [16]:
# Load SRA Run Info into scheduler ===================
# Scheduler DNS: 
echo "Loading SRARunInfo into scheduler "
echo "  File: $CURRENT_BATCH"
echo "  md5 : $(md5sum $CURRENT_BATCH)"
echo "  date: $(date)"

curl -s -X POST -T $CURRENT_BATCH localhost:8000/jobs/add_sra_run_info/

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200525_ab/vert_1.csv
  md5 : e49c0208b7b07074e1fd01f42cdba81b  /home/artem/serratus/notebook/200525_ab/vert_1.csv
  date: Tue May 26 15:55:26 PDT 2020
{"inserted_rows":20000,"total_rows":20000}


In [83]:
# Set Cluster Parameters =============================
# Scaled to 300:650:10
cd $TF
# Make local changes to config file
echo "  Cluster Config File: "
cat serratus-config.json
echo ""
echo ""
# Re-upload config file
curl -T serratus-config.json localhost:8000/config

  Cluster Config File: 
{
"ALIGN_ARGS":"--very-sensitive-local",
"ALIGN_SCALING_CONSTANT":0.15,
"ALIGN_SCALING_ENABLE":true,
"ALIGN_SCALING_MAX":400,
"CLEAR_INTERVAL":800,
"DL_ARGS":"",
"DL_SCALING_CONSTANT":0.1,
"DL_SCALING_ENABLE":true,
"DL_SCALING_MAX":200,
"GENOME":"cov3m",
"MERGE_ARGS":"",
"MERGE_SCALING_CONSTANT":0.1,
"MERGE_SCALING_ENABLE":true,
"MERGE_SCALING_MAX":10,
"SCALING_INTERVAL":250,
"VIRTUAL_ASG_MAX_INCREASE":10,
"VIRTUAL_SCALING_INTERVAL":30
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0{"ALIGN_ARGS":"--very-sensitive-local","ALIGN_SCALING_CONSTANT":0.15,"ALIGN_SCALING_ENABLE":true,"ALIGN_SCALING_MAX":400,"CLEAR_INTERVAL":800,"DL_ARGS":"","DL_SCALING_CONSTANT":0.1,"DL_SCALING_ENABLE":true,"DL_SCALING_MAX":200,"GENOME":"cov3m","MERGE_ARGS":"","MERGE_SCALING_CONSTANT":0.1,"MERGE_

In [24]:
CURRENT_BATCH=$WORK/vert_2.csv
echo $CURRENT_BATCH

/home/artem/serratus/notebook/200525_ab/vert_2.csv


In [25]:
# Load SRA Run Info into scheduler ===================
# Scheduler DNS: 
echo "Loading SRARunInfo into scheduler "
echo "  File: $CURRENT_BATCH"
echo "  md5 : $(md5sum $CURRENT_BATCH)"
echo "  date: $(date)"

curl -s -X POST -T $CURRENT_BATCH localhost:8000/jobs/add_sra_run_info/

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200525_ab/vert_2.csv
  md5 : 43044a323448a47e6edab675a1c44f8f  /home/artem/serratus/notebook/200525_ab/vert_2.csv
  date: Tue May 26 18:40:12 PDT 2020
{"inserted_rows":19999,"total_rows":39999}


In [26]:
CURRENT_BATCH=$WORK/vert_3.csv
echo $CURRENT_BATCH

/home/artem/serratus/notebook/200525_ab/vert_3.csv


In [27]:
# Load SRA Run Info into scheduler ===================
# Scheduler DNS: 
echo "Loading SRARunInfo into scheduler "
echo "  File: $CURRENT_BATCH"
echo "  md5 : $(md5sum $CURRENT_BATCH)"
echo "  date: $(date)"

curl -s -X POST -T $CURRENT_BATCH localhost:8000/jobs/add_sra_run_info/

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200525_ab/vert_3.csv
  md5 : 78b20a25210e896924474e8433565766  /home/artem/serratus/notebook/200525_ab/vert_3.csv
  date: Tue May 26 22:32:30 PDT 2020
{"inserted_rows":20000,"total_rows":59999}


### Batch 4+ (next day)

In [44]:
CURRENT_BATCH=$WORK/vert_4.csv
echo $CURRENT_BATCH

/home/artem/serratus/notebook/200525_ab/vert_4.csv


In [45]:
# Load SRA Run Info into scheduler ===================
# Scheduler DNS: 
echo "Loading SRARunInfo into scheduler "
echo "  File: $CURRENT_BATCH"
echo "  md5 : $(md5sum $CURRENT_BATCH)"
echo "  date: $(date)"

curl -s -X POST -T $CURRENT_BATCH localhost:8000/jobs/add_sra_run_info/

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200525_ab/vert_4.csv
  md5 : 627d7616fd2a224d65775cf6b8fb6d53  /home/artem/serratus/notebook/200525_ab/vert_4.csv
  date: Wed May 27 13:34:00 PDT 2020
{"inserted_rows":20000,"total_rows":20000}


In [50]:
CURRENT_BATCH=$WORK/vert_5.csv
echo $CURRENT_BATCH

/home/artem/serratus/notebook/200525_ab/vert_5.csv


In [51]:
# Load SRA Run Info into scheduler ===================
# Scheduler DNS: 
echo "Loading SRARunInfo into scheduler "
echo "  File: $CURRENT_BATCH"
echo "  md5 : $(md5sum $CURRENT_BATCH)"
echo "  date: $(date)"

curl -s -X POST -T $CURRENT_BATCH localhost:8000/jobs/add_sra_run_info/

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200525_ab/vert_5.csv
  md5 : c06a444644fd26173dd86be2b2e88981  /home/artem/serratus/notebook/200525_ab/vert_5.csv
  date: Wed May 27 15:59:36 PDT 2020
{"inserted_rows":14908,"total_rows":34908}


### Clean up Batch

In [68]:
cd $WORK
CURRENT_BATCH=$WORK/batchB_sraRunInfo.csv

echo $CURRENT_BATCH

/home/artem/serratus/notebook/200525_ab/batchB_sraRunInfo.csv


In [69]:
# Load SRA Run Info into scheduler ===================
# Scheduler DNS: 
echo "Loading SRARunInfo into scheduler "
echo "  File: $CURRENT_BATCH"
echo "  md5 : $(md5sum $CURRENT_BATCH)"
echo "  date: $(date)"

curl -s -X POST -T $CURRENT_BATCH localhost:8000/jobs/add_sra_run_info/

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200525_ab/batchB_sraRunInfo.csv
  md5 : eabffcf812931ae10678eb531473f674  /home/artem/serratus/notebook/200525_ab/batchB_sraRunInfo.csv
  date: Thu May 28 10:39:44 PDT 2020
{"inserted_rows":27432,"total_rows":27432}


In [81]:
cd $WORK
CURRENT_BATCH=$WORK/batchC_sraRunInfo.csv

echo $CURRENT_BATCH

/home/artem/serratus/notebook/200525_ab/batchC_sraRunInfo.csv


In [82]:
# Load SRA Run Info into scheduler ===================
# Scheduler DNS: 
echo "Loading SRARunInfo into scheduler "
echo "  File: $CURRENT_BATCH"
echo "  md5 : $(md5sum $CURRENT_BATCH)"
echo "  date: $(date)"

curl -s -X POST -T $CURRENT_BATCH localhost:8000/jobs/add_sra_run_info/

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200525_ab/batchC_sraRunInfo.csv
  md5 : 56081790257158acc391fa19992149d2  /home/artem/serratus/notebook/200525_ab/batchC_sraRunInfo.csv
  date: Thu May 28 18:41:01 PDT 2020
{"inserted_rows":12429,"total_rows":12429}


### Error Handling


In [ ]:
# Error fixes (manually help along)

# ssh <scheduler IPv4>
# sudo docker ps
# sudo docker exec -it <container> bash
# apt install sqlite3 awscli

### ACCESSION OPERATIONS

# Reset SPLITTING accessions to NEW
# sqlite3 instance/scheduler.sqlite 'UPDATE acc SET state = "new" WHERE state = "splitting";'

# Reset SPLIT_ERR accessions to NEW
# (repeated failures can be missing SRA data)
# sqlite3 instance/scheduler.sqlite 'UPDATE acc SET state = "new" WHERE state = "split_err";'

# Reset MERGE_ERR accessions to MERGE_WAIT
# sqlite3 instance/scheduler.sqlite 'UPDATE acc SET state = "merge_wait" WHERE state = "merge_err";'

# Clear DONE Accessions (ONLY ON COMPLETION)
# sqlite3 instance/scheduler.sqlite 'DELETE FROM acc WHERE state = "merge_done";'

### BLOCK OPERATIONS

# Reset FAIL blocks to NEW
# sqlite3 instance/scheduler.sqlite 'UPDATE blocks SET state = "new" WHERE state = "fail";'

# Reset ALIGNING blocks to NEW
# sqlite3 instance/scheduler.sqlite 'UPDATE blocks SET state = "new" WHERE state = "aligning";'

# Clear DONE blocks
# sqlite3 instance/scheduler.sqlite 'DELETE FROM blocks WHERE state = "done";'


## Shutting down procedures

Closing up shop.

In [28]:
# Dump the Scheduler SQLITE table to a local file
date
curl localhost:8000/db > \
  $WORK/vert_complete1.sqlite

Wed May 27 08:30:50 PDT 2020
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 29  202M   29 59.5M    0     0  1049k      0  0:03:17  0:00:58  0:02:19 1389k
curl: (18) transfer closed with 150202424 bytes remaining to read


In [ ]:
terraform destroy -auto-approve
# WARNING this will also delete the standard output bucket/data
# Save data prior to destroy

## Destroy Cluster

Close out all resources with terraform (will take a few minutes).


In [ ]:
# Finished all but ~400 stragler samples swing back around later for try-hard
#
#